# Dealing with imbalanced datasets, combining oversampling with VAE and undersampling to improve model recognition over all classes.  

Import packages, classifiers and etc.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from sklearn.metrics import confusion_matrix, make_scorer
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from collections import Counter

from imblearn.over_sampling import SMOTE, RandomOverSampler

Import VAEOversampler.

In [2]:
from VAEOversampler import VAEOversampler

2023-08-02 17:30:27.047932: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-02 17:30:27.047957: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading data  
You can load some dataset from Imbalanced Learn list (https://imbalanced-learn.org/stable/datasets/index.html) or use your own data.  


In [3]:
from imblearn.datasets import fetch_datasets

dset_name = 'abalone_19'
dset = fetch_datasets()[dset_name]

X, y = StandardScaler().fit_transform(dset.data), dset.target

In [4]:
X

array([[-0.67483383, -0.68801788,  1.31667716, ..., -0.60768536,
        -0.72621157, -0.63821689],
       [-0.67483383, -0.68801788,  1.31667716, ..., -1.17090984,
        -1.20522124, -1.21298732],
       [ 1.48184628, -0.68801788, -0.75948762, ..., -0.4634999 ,
        -0.35668983, -0.20713907],
       ...,
       [-0.67483383, -0.68801788,  1.31667716, ...,  0.74855917,
         0.97541324,  0.49695471],
       [ 1.48184628, -0.68801788, -0.75948762, ...,  0.77334105,
         0.73362741,  0.41073914],
       [-0.67483383, -0.68801788,  1.31667716, ...,  2.64099341,
         1.78744868,  1.84048058]])

In [5]:
y[y == -1] = 0

We split data into train and test partitions.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

This is a simple function to undersample freely.  

In [7]:
# RUS

def RUS(X_res, y_res, frac=1, minority_class_id=1, random_state=42):
    X_res = pd.DataFrame(X_res)
    X_res['Class'] = y_res  
    
    X_neg = X_res[y_res != minority_class_id].sample(frac=frac, random_state=random_state)
    X_pos = X_res[y_res == minority_class_id].sample(frac=1, random_state=random_state)
    
    X_rus = pd.concat([X_neg, X_pos], ignore_index=True)

    X_eq = X_rus.drop('Class', axis=1)
    y_eq = X_rus['Class']

    return X_eq, y_eq

In [8]:
def train_val(X, y, Xt, yt, random_state=42):
    classifiers = {
        "CatBoostClassifier": CatBoostClassifier(verbose=False, random_seed=random_state),
        "LGBMClassifier": LGBMClassifier(random_state=random_state),
        "XGBClassifier": XGBClassifier(random_state=random_state),
        "BaggingClassifier": BaggingClassifier(random_state=random_state),
        "RandomForestClassifier": RandomForestClassifier(random_state=random_state),
    }
    scores = []
    predictions = []
    for key, classifier in classifiers.items():
        print('_' * 50)
        name = key
        classifier.fit(X, y)
        print("Classifier: ", name)
        y_pred = classifier.predict(Xt)
        cm = confusion_matrix(yt, y_pred)
        print(cm)
        print('')
        predictions.append(y_pred)
        tn = cm[0,0]
        fp = cm[0,1]
        fn = cm[1,0]
        tp = cm[1,1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)
        scores.append(tnr * tpr)
        print('TNR:', round(tnr, 5))
        print('TPR:', round(tpr, 5))
        print('TNRxTPR:', round(tnr * tpr, 5))
        print('G-mean:', round(np.sqrt(tnr * tpr), 5))

    print('_' * 50)
    print('Ensemble predictions (majority voting):')
    predictions = np.sum(predictions, axis=0)
    predictions[predictions < 3] = 0
    predictions[predictions >= 3] = 1

    cm = confusion_matrix(yt, predictions)
    print(cm)
    tn = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    tp = cm[1,1]
    tnr = tn / (tn + fp)
    tpr = tp / (tp + fn)
    print('')
    print('TNR:', round(tnr, 5))
    print('TPR:', round(tpr, 5))
    print('TNRxTPR:', round(tnr * tpr, 5))
    print('G-mean:', round(np.sqrt(tnr * tpr), 5))
    

## Without resampling (base line) 
Which is starting score?


In [9]:
print('Original dataset shape %s' % Counter(y))
print('Ratio->', round(Counter(y)[0]/Counter(y)[1], 1), ': 1')

Original dataset shape Counter({0: 4145, 1: 32})
Ratio-> 129.5 : 1


In [10]:
train_val(X_train, y_train, X_train, y_train)

__________________________________________________
Classifier:  CatBoostClassifier
[[3315    0]
 [   9   17]]

TNR: 1.0
TPR: 0.65385
TNRxTPR: 0.65385
G-mean: 0.80861
__________________________________________________
Classifier:  LGBMClassifier
[[3315    0]
 [   0   26]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  XGBClassifier
[[3315    0]
 [   1   25]]

TNR: 1.0
TPR: 0.96154
TNRxTPR: 0.96154
G-mean: 0.98058
__________________________________________________
Classifier:  BaggingClassifier
[[3315    0]
 [   7   19]]

TNR: 1.0
TPR: 0.73077
TNRxTPR: 0.73077
G-mean: 0.85485
__________________________________________________
Classifier:  RandomForestClassifier
[[3315    0]
 [   0   26]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Ensemble predictions (majority voting):
[[3315    0]
 [   0   26]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0


In [11]:
train_val(X_train, y_train, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[830   0]
 [  6   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  LGBMClassifier
[[830   0]
 [  6   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  XGBClassifier
[[830   0]
 [  6   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  BaggingClassifier
[[830   0]
 [  6   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  RandomForestClassifier
[[830   0]
 [  6   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[830   0]
 [  6   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0


## Ratio 1:1  
Let's see classifiers scores when dataset is balanced.  


In [12]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

In [13]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 3315, 1: 3315})
Ratio->  1 : 1.0


In [14]:
train_val(X_res, y_res, X_res, y_res)

__________________________________________________
Classifier:  CatBoostClassifier
[[3315    0]
 [   1 3314]]

TNR: 1.0
TPR: 0.9997
TNRxTPR: 0.9997
G-mean: 0.99985
__________________________________________________
Classifier:  LGBMClassifier
[[3315    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  XGBClassifier
[[3315    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  BaggingClassifier
[[3315    0]
 [   3 3312]]

TNR: 1.0
TPR: 0.9991
TNRxTPR: 0.9991
G-mean: 0.99955
__________________________________________________
Classifier:  RandomForestClassifier
[[3315    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Ensemble predictions (majority voting):
[[3315    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0


In [15]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[816  14]
 [  6   0]]

TNR: 0.98313
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  LGBMClassifier
[[823   7]
 [  6   0]]

TNR: 0.99157
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  XGBClassifier
[[822   8]
 [  6   0]]

TNR: 0.99036
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  BaggingClassifier
[[807  23]
 [  5   1]]

TNR: 0.97229
TPR: 0.16667
TNRxTPR: 0.16205
G-mean: 0.40255
__________________________________________________
Classifier:  RandomForestClassifier
[[813  17]
 [  6   0]]

TNR: 0.97952
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[819  11]
 [  6   0]]

TNR: 0.98675
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0


In [16]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [17]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 3315, 1: 3315})
Ratio->  1 : 1.0


In [18]:
train_val(X_res, y_res, X_res, y_res)

__________________________________________________
Classifier:  CatBoostClassifier
[[3315    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  LGBMClassifier
[[3315    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  XGBClassifier
[[3315    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  BaggingClassifier
[[3315    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  RandomForestClassifier
[[3315    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Ensemble predictions (majority voting):
[[3315    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0


In [19]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[826   4]
 [  6   0]]

TNR: 0.99518
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  LGBMClassifier
[[829   1]
 [  6   0]]

TNR: 0.9988
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  XGBClassifier
[[827   3]
 [  6   0]]

TNR: 0.99639
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  BaggingClassifier
[[827   3]
 [  6   0]]

TNR: 0.99639
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  RandomForestClassifier
[[830   0]
 [  6   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[829   1]
 [  6   0]]

TNR: 0.9988
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0


In [20]:
# VAEOversampler

vae_sampler = VAEOversampler(epochs=500,
                              intermediate_dim=512,
                              batch_size=64,
                              random_state=42,
                              verbose=False)
X_res, y_res = vae_sampler.fit_resample(X_train, y_train)

2023-08-02 17:32:07.827698: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-08-02 17:32:07.827723: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-02 17:32:07.827741: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a-Modern-15-A5M): /proc/driver/nvidia/version does not exist
2023-08-02 17:32:07.827935: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0.0: 3315, 1.0: 3315})
Ratio->  1 : 1.0


In [22]:
train_val(X_res, y_res, X_res, y_res)

__________________________________________________
Classifier:  CatBoostClassifier
[[3315    0]
 [   9 3306]]

TNR: 1.0
TPR: 0.99729
TNRxTPR: 0.99729
G-mean: 0.99864
__________________________________________________
Classifier:  LGBMClassifier
[[3315    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  XGBClassifier
[[3315    0]
 [   1 3314]]

TNR: 1.0
TPR: 0.9997
TNRxTPR: 0.9997
G-mean: 0.99985
__________________________________________________
Classifier:  BaggingClassifier
[[3315    0]
 [   4 3311]]

TNR: 1.0
TPR: 0.99879
TNRxTPR: 0.99879
G-mean: 0.9994
__________________________________________________
Classifier:  RandomForestClassifier
[[3315    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Ensemble predictions (majority voting):
[[3315    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0


In [23]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[830   0]
 [  6   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  LGBMClassifier
[[830   0]
 [  6   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  XGBClassifier
[[830   0]
 [  6   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  BaggingClassifier
[[830   0]
 [  6   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  RandomForestClassifier
[[830   0]
 [  6   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[830   0]
 [  6   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0


## Under/Oversampling combination  
Now we can tuning the number of instances for each class to optimize metric.  


In [24]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

In [25]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.05)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 3315, 0: 166})
Ratio->  1 : 20.0


In [26]:
train_val(X_eq, y_eq, X_eq, y_eq)

__________________________________________________
Classifier:  CatBoostClassifier
[[ 164    2]
 [   0 3315]]

TNR: 0.98795
TPR: 1.0
TNRxTPR: 0.98795
G-mean: 0.99396
__________________________________________________
Classifier:  LGBMClassifier
[[ 166    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  XGBClassifier
[[ 166    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  BaggingClassifier
[[ 166    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  RandomForestClassifier
[[ 166    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Ensemble predictions (majority voting):
[[ 166    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0


In [27]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[608 222]
 [  0   6]]

TNR: 0.73253
TPR: 1.0
TNRxTPR: 0.73253
G-mean: 0.85588
__________________________________________________
Classifier:  LGBMClassifier
[[661 169]
 [  2   4]]

TNR: 0.79639
TPR: 0.66667
TNRxTPR: 0.53092
G-mean: 0.72865
__________________________________________________
Classifier:  XGBClassifier
[[658 172]
 [  2   4]]

TNR: 0.79277
TPR: 0.66667
TNRxTPR: 0.52851
G-mean: 0.72699
__________________________________________________
Classifier:  BaggingClassifier
[[655 175]
 [  3   3]]

TNR: 0.78916
TPR: 0.5
TNRxTPR: 0.39458
G-mean: 0.62815
__________________________________________________
Classifier:  RandomForestClassifier
[[642 188]
 [  2   4]]

TNR: 0.77349
TPR: 0.66667
TNRxTPR: 0.51566
G-mean: 0.7181
__________________________________________________
Ensemble predictions (majority voting):
[[650 180]
 [  2   4]]

TNR: 0.78313
TPR: 0.66667
TNRxTPR: 0.52209
G-mean: 0.72256


In [28]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [29]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.05)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 3315, 0: 166})
Ratio->  1 : 20.0


In [30]:
train_val(X_eq, y_eq, X_eq, y_eq)

__________________________________________________
Classifier:  CatBoostClassifier
[[ 163    3]
 [   0 3315]]

TNR: 0.98193
TPR: 1.0
TNRxTPR: 0.98193
G-mean: 0.99092
__________________________________________________
Classifier:  LGBMClassifier
[[ 166    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  XGBClassifier
[[ 166    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  BaggingClassifier
[[ 166    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  RandomForestClassifier
[[ 166    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Ensemble predictions (majority voting):
[[ 166    0]
 [   0 3315]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0


In [31]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[683 147]
 [  1   5]]

TNR: 0.82289
TPR: 0.83333
TNRxTPR: 0.68574
G-mean: 0.8281
__________________________________________________
Classifier:  LGBMClassifier
[[731  99]
 [  3   3]]

TNR: 0.88072
TPR: 0.5
TNRxTPR: 0.44036
G-mean: 0.6636
__________________________________________________
Classifier:  XGBClassifier
[[728 102]
 [  4   2]]

TNR: 0.87711
TPR: 0.33333
TNRxTPR: 0.29237
G-mean: 0.54071
__________________________________________________
Classifier:  BaggingClassifier
[[740  90]
 [  5   1]]

TNR: 0.89157
TPR: 0.16667
TNRxTPR: 0.14859
G-mean: 0.38548
__________________________________________________
Classifier:  RandomForestClassifier
[[767  63]
 [  4   2]]

TNR: 0.9241
TPR: 0.33333
TNRxTPR: 0.30803
G-mean: 0.55501
__________________________________________________
Ensemble predictions (majority voting):
[[745  85]
 [  4   2]]

TNR: 0.89759
TPR: 0.33333
TNRxTPR: 0.2992
G-mean: 0.54699


In [32]:
# VAEOversampler

X_res, y_res = vae_sampler.resample(X_train, y_train, sampling_strategy=2)

In [33]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.0155)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1.0: 6604, 0.0: 51})
Ratio->  1 : 129.5


In [34]:
train_val(X_eq, y_eq, X_eq, y_eq)

__________________________________________________
Classifier:  CatBoostClassifier
[[  50    1]
 [   1 6603]]

TNR: 0.98039
TPR: 0.99985
TNRxTPR: 0.98024
G-mean: 0.99007
__________________________________________________
Classifier:  LGBMClassifier
[[  50    1]
 [   1 6603]]

TNR: 0.98039
TPR: 0.99985
TNRxTPR: 0.98024
G-mean: 0.99007
__________________________________________________
Classifier:  XGBClassifier
[[  51    0]
 [   1 6603]]

TNR: 1.0
TPR: 0.99985
TNRxTPR: 0.99985
G-mean: 0.99992
__________________________________________________
Classifier:  BaggingClassifier
[[  51    0]
 [   3 6601]]

TNR: 1.0
TPR: 0.99955
TNRxTPR: 0.99955
G-mean: 0.99977
__________________________________________________
Classifier:  RandomForestClassifier
[[  51    0]
 [   0 6604]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Ensemble predictions (majority voting):
[[  51    0]
 [   1 6603]]

TNR: 1.0
TPR: 0.99985
TNRxTPR: 0.99985
G-mean: 0.99992


In [35]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[665 165]
 [  3   3]]

TNR: 0.8012
TPR: 0.5
TNRxTPR: 0.4006
G-mean: 0.63293
__________________________________________________
Classifier:  LGBMClassifier
[[653 177]
 [  2   4]]

TNR: 0.78675
TPR: 0.66667
TNRxTPR: 0.5245
G-mean: 0.72422
__________________________________________________
Classifier:  XGBClassifier
[[680 150]
 [  4   2]]

TNR: 0.81928
TPR: 0.33333
TNRxTPR: 0.27309
G-mean: 0.52258
__________________________________________________
Classifier:  BaggingClassifier
[[700 130]
 [  1   5]]

TNR: 0.84337
TPR: 0.83333
TNRxTPR: 0.70281
G-mean: 0.83834
__________________________________________________
Classifier:  RandomForestClassifier
[[693 137]
 [  2   4]]

TNR: 0.83494
TPR: 0.66667
TNRxTPR: 0.55663
G-mean: 0.74607
__________________________________________________
Ensemble predictions (majority voting):
[[680 150]
 [  2   4]]

TNR: 0.81928
TPR: 0.66667
TNRxTPR: 0.54618
G-mean: 0.73904


## References  

  - Classification with Imbalanced Datasets:  
    https://sci2s.ugr.es/imbalanced  
  - Computer Vision:  Models, Learning, and Inference (Simon J.D. Prince):  
    http://www.computervisionmodels.com/  
  - Oversampling with VAEs:  
    https://towardsdatascience.com/oversampling-with-vaes-e410887fe51  
